In [1]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import (
    run_full_analysis, 
    print_continuous_status, 
    download_analysis,
    DEFAULT_RUN_CONFIG,
    TEST_RUN_CONFIG,
)

c:\Users\rrag0004\.conda\envs\summer2\lib\site-packages\summer\runner\vectorized_runner.py:363: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_strain_infection_values(
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Define task function

#### Standard config

In [2]:
DEFAULT_RUN_CONFIG

{'n_cores': 8,
 'n_opti_searches': 16,
 'opti_budget': 10000,
 'n_chains': 8,
 'metropolis_tune': 5000,
 'metropolis_draws': 30000,
 'metropolis_method': 'DEMetropolisZ',
 'full_runs_samples': 1000,
 'burn_in': 20000}

In [11]:
run_config = DEFAULT_RUN_CONFIG | {'n_cores': 4}  # update if required

In [12]:
run_config

{'n_cores': 4,
 'n_opti_searches': 16,
 'opti_budget': 10000,
 'n_chains': 8,
 'metropolis_tune': 5000,
 'metropolis_draws': 30000,
 'metropolis_method': 'DEMetropolisZ',
 'full_runs_samples': 1000,
 'burn_in': 20000}

In [13]:
ISO3 = "FRA"
ANALYSIS = "main"

In [14]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        run_config=run_config,
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [15]:
mspec = springboard.EC2MachineSpec(run_config['n_cores'], 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [16]:
analysis_title = f"refactored_{run_config['n_cores']}cpu"
config_str = f"_{ANALYSIS}_LHS{run_config['n_opti_searches']}_opt{run_config['opti_budget']}_mc{run_config['metropolis_tune']}n{run_config['metropolis_draws']}"

run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, analysis_title + config_str)
run_path

'projects/school_project/FRA/2023-09-18T1101-refactored_4cpu_main_LHS16_opt10000_mc5000n30000'

In [17]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

SSH connection still waiting, retrying 1/5
SSH connection still waiting, retrying 2/5


In [19]:
runner.s3.get_status()

'RUNNING'

In [ ]:
print(runner.top("%MEM"))

In [ ]:
# wait function with status printing
# print_continuous_status(runner)

In [20]:
run_path = 'projects/school_project/FRA/2023-09-18T1059-refactored_8cpu_main_LHS16_opt10000_mc5000n30000'

In [21]:
download_analysis(run_path)